<a href="https://colab.research.google.com/github/pranavkarnani/StoryGenerator/blob/pranav/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 8.9 MB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! unzip -q /content/plots.zip

In [4]:
! pip install rake_nltk

     |████████████████████████████████| 1.5 MB 8.1 MB/s 
     |████████████████████████████████| 749 kB 47.1 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from rake_nltk import Rake
from nltk.tokenize import sent_tokenize
import os
from tqdm import tqdm

infile = '/content/plots'
infile_title = '/content/titles'
outfile = '/content/wikiplot.kwRAKE.tsv'

f = open(infile, 'r', encoding='"ISO-8859-1"')
f_title = open(infile_title, 'r', encoding='"ISO-8859-1"')
fout = open(outfile, 'a', encoding='"ISO-8859-1"')

lines = f.readlines()
lines_title = f_title.readlines()

sentences_to_write = []
w = 0
total = 0
sentences_to_write.append("[ID]\t[KEY/ABSTRACT]\t[KEYWORDS]\t[DISCOURSE (T/I/B/C)]\t[NUM_PARAGRAPHS]\t[PARAGRAPH]\t[PREVIOUS_PARAGRAPH]\n")

title_idx = 0
line_idx = 0
my_lines = {}

prev = 0
for line_idx, line in enumerate(tqdm(lines)):
    if "<EOS>" in line:
        article = "".join(lines[prev:line_idx])
        my_lines[title_idx] = article
        prev = line_idx + 1
        title_idx += 1

100%|██████████| 2070449/2070449 [00:01<00:00, 1368642.89it/s]


In [3]:
my_lines

{0: 'Old Major, the old boar on the Manor Farm, summons the animals on the farm together for a meeting, during which he refers to humans as "enemies" and teaches the animals a revolutionary song called "Beasts of England".\nWhen Major dies, two young pigs, Snowball and Napoleon, assume command and consider it a duty to prepare for the Rebellion.\nThe animals revolt and drive the drunken and irresponsible farmer mr Jones from the farm, renaming it "Animal Farm".\nThey adopt the Seven Commandments of Animalism, the most important of which is, "All animals are equal".\nSnowball teaches the animals to read and write, while Napoleon educates young puppies on the principles of Animalism.\nFood is plentiful, and the farm runs smoothly.\nThe pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health.\nSome time later, several men attack Animal Farm.\nJones and his men are making an attempt to recapture the farm, aided by several ot

In [4]:
import pandas as pd
df = pd.DataFrame()

In [5]:
df['id'] = my_lines.keys()

In [6]:
df['text'] = my_lines.values()

In [7]:
df['title'] = lines_title

In [8]:
df['title'] = df['title'].apply(lambda x: x[:-1])

In [9]:
df

,id,text,title
0,0,"Old Major, the old boar on the Manor Farm, sum...",Animal Farm
1,1,Alex is a 15-year-old living in near-future dy...,A Clockwork Orange (novel)
2,2,The text of The Plague is divided into five pa...,The Plague
3,3,"Among others, John Heath has observed, ""The un...",Actaeon
4,4,"An expedition from Straumli Realm, an ambitiou...",A Fire Upon the Deep
...,...,...,...
112931,112931,What starts as a clash of personalities turns ...,Downwardly Mobile
112932,112932,Etheria is the oldest and most powerful kingdo...,Etheria (2017 TV series)
112933,112933,Anna Sinclair is an established and in-demand ...,Anna In-Between
112934,112934,"In March 2012, five crew members of a news cha...",Bhangarh: The Last Episode


In [28]:
r = Rake()

In [31]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
for i in tqdm(range(len(lines_title))):
    text = df.loc[i, 'text']
    r.extract_keywords_from_text(text)
    df.loc[i,'outline'] = "##".join(r.get_ranked_phrases()[0:10])

100%|██████████| 112936/112936 [08:47<00:00, 214.06it/s]


In [37]:
df

,id,text,title,outline
0,0,"Old Major, the old boar on the Manor Farm, sum...",Animal Farm,set aside special food items##happiest animals...
1,1,Alex is a 15-year-old living in near-future dy...,A Clockwork Orange (novel),clad young woman whose presence##corrective ad...
2,2,The text of The Plague is divided into five pa...,The Plague,two men go swimming together##official notices...
3,3,"Among others, John Heath has observed, ""The un...",Actaeon,"least four attic tragedies##hero archegetes (""..."
4,4,"An expedition from Straumli Realm, an ambitiou...",A Fire Upon the Deep,young boy named jefri olsndot##device help woo...
...,...,...,...,...
112931,112931,What starts as a clash of personalities turns ...,Downwardly Mobile,wife sophie move##psychotherapist wife rosemar...
112932,112932,Etheria is the oldest and most powerful kingdo...,Etheria (2017 TV series),four kingdoms helped cassiopea defeat etheria#...
112933,112933,Anna Sinclair is an established and in-demand ...,Anna In-Between,vacation trip getting away##race plays vital r...
112934,112934,"In March 2012, five crew members of a news cha...",Bhangarh: The Last Episode,paranormal tv show##news channel disappeared##...


In [39]:
df.loc[0,'outline']

'set aside special food items##happiest animals live simple lives##several men attack animal farm##young pig named squealer##napoleon educates young puppies##dogs chase snowball away##irresponsible farmer mr jones##boxer continues working harder##pig ", notices##using blasting powder'

In [42]:
df.to_csv('data_20line_outline.csv')

In [41]:
for i in tqdm(range(len(lines_title))):
    text = df.loc[i, 'text']
    r.extract_keywords_from_text(text)
    df.loc[i,'outline'] = "##".join(r.get_ranked_phrases()[0:20])

100%|██████████| 112936/112936 [11:16<00:00, 166.93it/s]


In [43]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import csv

In [58]:
class StoryOutlineDataset(Dataset):

    def __init__(self, data_path, partition = "train"):

        df = pd.read_csv(data_path)

        if partition == "train":
            sample = len(df)*80//100
        else:
            sample = len(df) - len(df)*80//100 + 1

        self.X = df.loc[list(range(sample)), ['title', 'outline']]
        self.Y = df.loc[list(range(sample)), ['text']]

    def __len__(self):
        self.length = len(self.Y)
        return self.length

    def __getitem__(self, ind):
        title = str(self.X.loc[ind, 'title'])
        outline = str(self.X.loc[ind, 'outline']).split("##")
        target = str(self.Y.loc[ind,'text'])
        return ((title, outline), target)

In [59]:
batch_size = 32

In [60]:
train_data = StoryOutlineDataset('/content/data_20line_outline.csv', 'train')
val_data = StoryOutlineDataset('/content/data_20line_outline.csv', 'dev')

train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)


In [63]:
train_data.__getitem__(2)

(('The Plague',
  ['two men go swimming together',
   'official notices enacting control measures',
   'two government employees approach',
   'later informed via telegram',
   'hysteria develops soon afterward',
   'actor portraying orpheus collapses',
   'filled within three days',
   'disease still proves fatal',
   'separation affects daily activity',
   'plague serum finally arrives',
   'new antiplague serum',
   'approach fellow doctors',
   'waste away emotionally',
   'public pressure order',
   'leaving short telegrams',
   'inhabitants passively endure',
   'declaring martial law',
   'affect various characters',
   'feel less separated',
   'since everybody faces']),
 'The text of The Plague is divided into five parts.\nIn the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets.\nHysteria develops soon afterward, causing the local newspapers to report the incident.\nAuthorities responding to public pressure order the collection 